In [1]:
import tensorflow as tf
import chess.pgn
import pandas as pd
import numpy as np
import os
import time
import multiprocessing as mp
import random
import io
import re


2025-05-26 20:43:26.022656: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748313806.037152   10434 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748313806.041482   10434 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-26 20:43:26.056697: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

try:
    if gpus:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print('Now using gpu')
    else: print('Unable to use the gpu')


except RuntimeError as e:
    print(f'Something went wrong: {e}')


Now using gpu


In [3]:
file = '../data/raw/lichess_db_standard_rated_2016-03.pgn'

In [5]:
# def chess_pipeline(pgn_file, max_games = None):
#     start = time.time()
#     data = []
#     with open(pgn_file) as pgn:
#         fmt_game = 0
#         while True:

#             game = chess.pgn.read_game(pgn)
#             if game is None:
#                 print('No data found')
#                 break
#             game_parsed = game_pipeline(game)
#             data.append(game_parsed)
#             fmt_game += 1
#             if max_games is not None and fmt_game >= max_games:
#                 break
#             elif fmt_game % 1000 == 0:
#                 print(f'{fmt_game} games transformed')
#     end = time.time()
#     print(f'Time elapsed: {end-start:.2f} seconds')
#     return list(filter(None,data))



# def game_pipeline(game):
#     game_list = []
#     board = game.board()
#     result = game.headers['Result']
#     outcome = 1 if result == '1-0' else -1 if result == '0-1' else 0
#     moves = list(game.mainline_moves())
#     if moves:
#         for move in moves:
#             fen = board.fen()
#             move_uci = move.uci()
#             move_san = board.san(move)
#             player = 'white' if board.turn == chess.WHITE else 'black'

#             game_list.append(
#                 {
#                     'fen': fen,
#                     'move_uci': move_uci,
#                     'move_san': move_san,
#                     'player': player,
#                     'result': outcome
#                 })
#             board.push(move)
#     return game_list

In [ ]:
def game_pipeline_from_text(game_text):
    game = chess.pgn.read_game(io.StringIO(game_text))
    if game is None:
        return None

    moves = list(game.mainline_moves())

    game_list = []
    board = game.board()
    result = game.headers['Result']
    outcome = 1 if result == '1-0' else -1 if result == '0-1' else 0

    for move in moves:
        fen = board.fen()
        move_uci = move.uci()
        move_san = board.san(move)
        player = 'white' if board.turn == chess.WHITE else 'black'
        game_list.append({
            'fen': fen,
            'move_uci': move_uci,
            'move_san': move_san,
            'player': player,
            'result': outcome
        })
        board.push(move)

    return game_list if game_list else None

def process_and_save_chunk(games_chunk, chunk_idx):
    results = list(filter(None, map(lambda g: game_pipeline_from_text(g), games_chunk)))
    flat_results = [entry for game in results for entry in game]  # Flatten list
    df = pd.DataFrame(flat_results)
    df.to_csv(f"../data/csv/processed_chunk_{chunk_idx}.csv", index=False)
    print(f"Saved chunk {chunk_idx} with {len(results)} valid games")

    
def get_completed_chunk_ids(path='../data/csv/'):

    completed_chunks = os.listdir(path)
    completed_chunks_ids = [int(re.search(r'\d+', chunk)[0]) for chunk in completed_chunks]
    return completed_chunks_ids

chunk_size = 10_000
game_idx = 0
games_chunk = []
completed_chunks_ids = get_completed_chunk_ids()

with open(file) as f:
    while True:
        game = chess.pgn.read_game(f)
        if game is None:
            break

        current_chunk_idx = game_idx // chunk_size

        if current_chunk_idx in completed_chunks_ids:
            game_idx +=1
            continue
        games_chunk.append(str(game))
        game_idx += 1

        if len(games_chunk) >= chunk_size:
            print(f'Processing chunk {current_chunk_idx} ')
            process_and_save_chunk(games_chunk, current_chunk_idx)  # usa un número razonable o calcula primero
            games_chunk = []


# Procesar último chunk
if games_chunk:
    current_chunk_idx = game_idx // chunk_size
    print(f"Processing final chunk {current_chunk_idx}...")
    process_and_save_chunk(games_chunk, current_chunk_idx)


In [28]:

def format_dataframe(dataset):
    all_moves = [
        {**move, 'game_id': i}
        for i, game in enumerate(dataset)
        for move in game
    ]
    return pd.DataFrame(all_moves)


In [10]:
502320//10000

50

In [39]:
!snakeviz -s formatter.prof




snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2Fmnt%2Fc%2FUsers%2Frodri%2FDocuments%2FCareer%20Learning%2FDataLab%2Fchess_ai_project%2Fnotebooks%2Fformatter.prof
^C

Bye!
